The World Register of Marine Species (WoRMS) is another taxonomic authority we use in our work. In the case of the FWS work plan species, we mostly rely on ITIS as that is the taxonomic authority most used by FWS, and ITIS TSNs are determined/declared for most species. This notebook follows the ITIS caching process to retrieve any unmatched names and try them against WoRMS using the worms module of the bispy package.

In [1]:
import requests
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed
import random

worms = bispy.worms.Worms()

In [2]:
with open("cache/itis.json", "r") as f:
    itis_cache = json.loads(f.read())

In [4]:
#Unwind the ITIS URLs into names
unmatched_itis_names = [i["Processing Metadata"]["Detailed Results"][0]["Exact Match Fail"].replace("http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:", "").replace("\\%20", " ") for i in itis_cache if i["Processing Metadata"]["Summary Result"] == "Not Matched"]


In [5]:
# Use joblib to run multiple searches for WoRMS species in parallel via species names
worms_result = Parallel(n_jobs=8)(delayed(worms.search)(name) for name in unmatched_itis_names)

In [7]:
# We're just going to use the exact match records from WoRMS at this point
worms_cache = [w for w in worms_result if w["Processing Metadata"]["Summary Result"] == "Exact Match"]

In [8]:
# Dump the WoRMS data to a cache file
with open("cache/worms.json", "w") as f:
    f.write(json.dumps(worms_cache, indent=4))

In [9]:
# Open the file back up and verify
with open("cache/worms.json", "r") as f:
    worms_cache = json.loads(f.read())

print(len(worms_cache))
display(worms_cache[random.randint(0,len(worms_cache)-1)])

24


{'Processing Metadata': {'Date Processed': '2019-07-01T19:08:16.587424',
  'Search URL': 'http://www.marinespecies.org/rest/AphiaRecordsByName/Pyrgulopsis coloradensis?like=false&marine_only=false&offset=1',
  'Status': 'Error',
  'Summary Result': 'Exact Match'},
 'wormsData': [{'AphiaID': 1346746,
   'authority': 'Hershler, 1998',
   'citation': 'MolluscaBase (2019). MolluscaBase. Pyrgulopsis coloradensis Hershler, 1998. Accessed through: World Register of Marine Species at: http://www.marinespecies.org/aphia.php?p=taxdetails&id=1346746 on 2019-07-01',
   'class': 'Gastropoda',
   'family': 'Hydrobiidae',
   'genus': 'Pyrgulopsis',
   'isBrackish': 0,
   'isExtinct': 0,
   'isFreshwater': 1,
   'isMarine': 0,
   'isTerrestrial': 0,
   'kingdom': 'Animalia',
   'lsid': 'urn:lsid:marinespecies.org:taxname:1346746',
   'match_type': 'exact',
   'modified': '2019-05-16T06:59:02.523Z',
   'order': 'Littorinimorpha',
   'parentNameUsageID': 717226,
   'phylum': 'Mollusca',
   'rank': 'Spec